In [3]:
import numpy as np
from model_functions import *
from Classification_models_LOSO_2Class import *

In [ ]:
#uncomment to run and create the all_psds file. 

#subject_data, freqs, targets = load_data(duration=30,overlap=15,seg_length=4)
#save_psds(subject_data,freqs,targets,'all_psds.pickle')


In [4]:
subject_data, freqs, targets = load_psds('all_psds.pickle')
test_labels = align_test_labels(classes=['A','C'])
features, targets = remove_class(subject_data,targets,'F')
features, targets = remove_test(features,targets,test_labels)
rbps = [relative_band_power(features[i],freqs,freq_bands=[0.5,4.0,7.0,9.0,11.0,13.0,25.0,45.0]) 
        for i in range(len(features))]

### k nearest neighbor classifier

In [ ]:
train_metric_dict, val_metric_dict = kNN_cross(rbps, targets, n_neighbors = 3)

In [ ]:
train_metric_dict, val_metric_dict

### Random Forest classifier

Here is a run of random Forest without doing PCA. As you can see in the definition of the method, PCA = 0, means we do not do PCA

In [5]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 0)
train_metric_dict, val_metric_dict

({'acc': 0.9912203841300995,
  'sens': 0.995752427184466,
  'spec': 0.9859195984994423,
  'f1': 0.9918886100542379},
 {'acc': 0.745829244357213,
  'sens': 0.7785194174757282,
  'spec': 0.7075940383250532,
  'f1': 0.7675740352976368})

Here is a run of random Forest with PCA. The maximum number of features is 133 here. 
As you can see, there is a significant reduction in validation set accuracy (from 75% to 71% )

In [6]:
train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = 133)
train_metric_dict, val_metric_dict

({'acc': 0.9990887424646011,
  'sens': 0.9998266296809986,
  'spec': 0.9982256919801278,
  'f1': 0.9991553871142393},
 {'acc': 0.7101733725875041,
  'sens': 0.7536407766990292,
  'spec': 0.6593328601845281,
  'f1': 0.7370919881305638})

In [ ]:
PCA_components = [5, 10, 20, 40, 60, 80, 100, 133]
val_acc = []
train_acc = []
for n in PCA_components: 
    train_metric_dict, val_metric_dict = RF_cross(rbps, targets, n_estimators = 100, 
                                              min_samples_split = 16, PCA_components = n)
    val_acc += [val_metric_dict['acc']]
    train_acc += [train_metric_dict['acc']]